In [1]:
import pandas as pd
import matplotlib.pyplot as plt

outcome_df = pd.read_csv('data/new_new_outcome.txt')

outcome_df['college_bt'].unique()

# convert terms that end in 25 or 40 to end with 30
def convert_terms(major_history):
    terms = major_history.split('|')
    converted_terms = []
    for term in terms:
        if '25' in term or '40' in term:
            converted_terms.append(term.replace('25', '30').replace('40', '30'))
        else:
            converted_terms.append(term)
    return '|'.join(converted_terms)

# Update the major_history column
outcome_df['major_history'] = outcome_df['major_history'].apply(convert_terms)


In [2]:
print("NaN Values Here")
#errors because so many NaN values (ask about this)
nan_rows = outcome_df[outcome_df['student_type_code'].isna()]
print(nan_rows.shape[0])  # Number of rows where 'student_type_code' is NaN
print(nan_rows)

nan_rows.groupby('term').count()

NaN Values Here
31875
       encrypt_id    term  transfer_hours_bt  total_sfa_hours_bt  \
1        30130517  202220               85.0                36.0   
2        30130517  202210               85.0                24.0   
3        30130199  202510               11.0               114.0   
8        30130199  202220                8.0                44.0   
9        30130199  202210                0.0                29.0   
...           ...     ...                ...                 ...   
73973        2882  202210               24.0                86.0   
73974        2154  202510                0.0                92.0   
73980        2154  202230                0.0                43.0   
73981        2154  202220                0.0                27.0   
73982        2154  202210                0.0                18.0   

       total_sfa_hours_et  begin_of_semester_gpa  semester_gpa  \
1                    48.0                   4.00          4.00   
2                    36.0    

,encrypt_id,transfer_hours_bt,total_sfa_hours_bt,total_sfa_hours_et,begin_of_semester_gpa,semester_gpa,end_of_term_gpa,college_bt,department_bt,major_bt,...,major_history,grad_term,grad_program,advisor_name,total_course_hours_attempted,withdrew,academic_standing,student_type_code,probation,discontinued
term,,,,,,,,,,,,,,,,,,,,,
202210,10216,10216,10216,10216,7443,10208,10182,10216,10216,10216,...,10216,869,869,2696,10216,10216,10208,0,10216,10216
202220,8971,8971,8971,8971,8534,8963,8956,8971,8971,8971,...,8971,1172,1172,2726,8971,8971,8963,0,8971,8971
202230,3359,3359,3359,3359,3112,3355,3357,3359,3359,3359,...,3359,228,228,1047,3359,3359,3355,0,3359,3359
202310,2,2,2,2,1,2,2,2,2,2,...,2,0,0,2,2,2,2,0,2,2
202320,2,2,2,2,1,2,2,2,2,2,...,2,0,0,2,2,2,2,0,2,2
202410,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,0,1,1
202420,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,0,1,1
202510,9323,9323,9323,9323,6326,29,6326,6551,6551,6551,...,9323,742,742,9302,9323,9323,6255,0,9323,9323


In [3]:
# make new column with apply lam - gives classification by credit hours (or binning)
outcome_df['total_hours'] = outcome_df['transfer_hours_bt'] + outcome_df['total_sfa_hours_bt']
#print(outcome_df['total_hours'])

#function that classfies students based on the number of credit hours
def classify_student(total_hours):
    if total_hours < 30:
        return 'Freshman'
    elif 30 <= total_hours < 60:
        return 'Sophomore'
    elif 60 <= total_hours < 90:
        return 'Junior'
    else:
        return 'Senior'

#for every student id apply the classification function
outcome_df['Classification'] = outcome_df['total_hours'].apply(classify_student)
#print(outcome_df['Classification'])



# Define a function to classify GPA based on end_of_term_gpa
def classify_gpa(gpa):
    if gpa < 2.0:
        return '2.0 or less'
    elif 2.0 <= gpa < 3.0:
        return '2.0-3.0'
    elif 3.0 <= gpa < 3.5:
        return '3.0-3.5'
    elif 3.5 <= gpa <= 4.0:
        return '3.5-4.0'
    else:
        return 'Invalid GPA'  # For values outside the 0-4.0 range

# Apply the function to create the GPA_Group column
outcome_df['GPA_Group'] = outcome_df['end_of_term_gpa'].apply(classify_gpa)

# Find rows with 'Invalid GPA' in the GPA_Group column
invalid_gpa_rows = outcome_df[outcome_df['GPA_Group'] == 'Invalid GPA']
print("Rows with Invalid GPA values:")
print(invalid_gpa_rows[['encrypt_id', 'end_of_term_gpa']])

# Drop rows with 'Invalid GPA'
outcome_df = outcome_df[outcome_df['GPA_Group'] != 'Invalid GPA']

Rows with Invalid GPA values:
       encrypt_id  end_of_term_gpa
36       30112192              NaN
56       30101509              NaN
63       30097971              NaN
74       30086018              NaN
137      30071479              NaN
...           ...              ...
73877       39750              NaN
73943       12727              NaN
73951       11444              NaN
73963        6068              NaN
73964        4405              NaN

[3294 rows x 2 columns]


In [4]:
import numpy as np

# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is either '202420' or '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students using function given
disc = outcome_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc['all_disc_rate'] = (disc['discontinued'] / disc['encrypt_id'] * 100).round(2)  # Round to 2 decimal places

#print dataframe with disc rate for all students
print("The discontinued rate for any college regardless of college switches.")
print(disc)

# Set the change column based on college_bt and college_et (considering only those who switched colleges)
outcome_df['change'] = np.where(
    outcome_df['college_bt'].isna() | outcome_df['college_et'].isna(), 0,  # No change if either is NaN
    np.where(outcome_df['college_bt'] != outcome_df['college_et'], 1, 0)  # Compare normally otherwise
)

# Print rows where 'change' is equal to 1 (students who switched colleges)
changed_rows = outcome_df[outcome_df['change'] == 1]
print(changed_rows[['encrypt_id', 'college_bt', 'college_et', 'change']])

# Discontinued rate for only those who switched colleges
print("The discontinued rate of only those who switched colleges.")
changed_students_df = outcome_df[outcome_df['change'] == 1]

# Calculate discontinued rate for students who switched colleges
changed_students_df = changed_students_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
changed_students_df['change_disc_rate'] = (changed_students_df['discontinued'] / changed_students_df['encrypt_id'] * 100).round(2)
print(changed_students_df)

print("Both the data for ALL Students within a college and ONLY those who switched colleges.")
# Merge the two DataFrames on 'college_et' and 'term'
merged_disc = pd.merge(disc, changed_students_df, on=['college_et', 'term'], how='left')
merged_disc = merged_disc[['all_disc_rate', 'change_disc_rate']]
print(merged_disc)

The discontinued rate for any college regardless of college switches.
                   encrypt_id  discontinued  all_disc_rate
college_et term                                           
BU         202210        1607           165          10.27
           202220        1399           161          11.51
           202230         619            57           9.21
           202310        1466           141           9.62
           202320        1287           142          11.03
           202330         569            47           8.26
           202410        1401           126           8.99
           202420        1260           151          11.98
           202430         521            32           6.14
ED         202210        2594           192           7.40
           202220        2327           207           8.90
           202230        1033            44           4.26
           202310        2430           181           7.45
           202320        2173           194  

In [5]:
import numpy as np
import pandas as pd

# Create the total hours column
outcome_df['total_hours'] = outcome_df['transfer_hours_bt'] + outcome_df['total_sfa_hours_bt']

# Function that classifies students based on the number of credit hours
def classify_student(total_hours):
    if total_hours < 30:
        return 'Freshman'
    elif 30 <= total_hours < 60:
        return 'Sophomore'
    elif 60 <= total_hours < 90:
        return 'Junior'
    else:
        return 'Senior'

# Apply the classification function
outcome_df['Classification'] = outcome_df['total_hours'].apply(classify_student)

# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is either '202420' or '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students
disc = outcome_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc['all_disc_rate'] = (disc['discontinued'] / disc['encrypt_id'] * 100).round(2)

# Set the change column to indicate college switch status
outcome_df['change'] = np.where(
    outcome_df['college_bt'].isna() | outcome_df['college_et'].isna(), 0,  # No change if either is NaN
    np.where(outcome_df['college_bt'] != outcome_df['college_et'], 1, 0)  # Compare otherwise
)

# Filter rows for students who switched colleges
changed_rows = outcome_df[outcome_df['change'] == 1]

# Calculate discontinued rate for students who switched colleges
changed_students_df = changed_rows.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
changed_students_df['change_disc_rate'] = (changed_students_df['discontinued'] / changed_students_df['encrypt_id'] * 100).round(2)

# Merge the discontinued rate data for all students and only those who switched colleges
merged_disc = pd.merge(disc, changed_students_df, on=['college_et', 'term'], how='left')

# Select only the relevant discontinued rate columns for display
merged_disc = merged_disc[['all_disc_rate', 'change_disc_rate']]
print("Discontinued rates for all students and only those who switched colleges:")
print(merged_disc)


Discontinued rates for all students and only those who switched colleges:
                   all_disc_rate  change_disc_rate
college_et term                                   
BU         202210          10.27              6.35
           202220          11.51             16.67
           202230           9.21             20.83
           202310           9.62             18.56
           202320          11.03             14.29
           202330           8.26              5.88
           202410           8.99             18.00
           202420          11.98             31.40
           202430           6.14             13.64
ED         202210           7.40             10.39
           202220           8.90             15.94
           202230           4.26              3.77
           202310           7.45              7.84
           202320           8.93             13.18
           202330           5.50              9.09
           202410           6.55              5.37
        

In [6]:
# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students by classification
disc_by_classification = outcome_df.groupby(['Classification', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc_by_classification['disc_rate'] = (disc_by_classification['discontinued'] / disc_by_classification['encrypt_id'] * 100).round(2)
print(disc_by_classification['encrypt_id'])

# Create a pivot table for discontinued rate by classification and term
disc_pivot = disc_by_classification.pivot_table(values='disc_rate', index='term', columns='Classification')
print("Discontinued rates by classification and term:")
print(disc_pivot)

Classification  term  
Freshman        202210    2780
                202220    1905
                202230     384
                202310    2956
                202320    2114
                202330     385
                202410    3049
                202420    2109
                202430     296
Junior          202210    2494
                202220    2344
                202230     978
                202310    2248
                202320    2134
                202330     881
                202410    2134
                202420    2017
                202430     739
Senior          202210    3020
                202220    2952
                202230    1464
                202310    2846
                202320    2786
                202330    1303
                202410    2619
                202420    2574
                202430    1234
Sophomore       202210    1888
                202220    1755
                202230     531
                202310    1659
                

In [7]:
print("These represent the discontinued rate given GPA groups in general by term.")


# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students by classification
disc_by_classification = outcome_df.groupby(['GPA_Group', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc_by_classification['disc_rate'] = (disc_by_classification['discontinued'] / disc_by_classification['encrypt_id'] * 100).round(2)
print(disc_by_classification['encrypt_id'])

# Create a pivot table for discontinued rate by classification and term
disc_pivot = disc_by_classification.pivot_table(values='disc_rate', index='term', columns='GPA_Group')
print("Discontinued rates by GPA Group and term:")
print(disc_pivot)

These represent the discontinued rate given GPA groups in general by term.
GPA_Group    term  
2.0 or less  202210     802
             202220     599
             202230     196
             202310     722
             202320     539
             202330     174
             202410     879
             202420     767
             202430     273
2.0-3.0      202210    3595
             202220    3275
             202230    1373
             202310    3396
             202320    3179
             202330    1253
             202410    3335
             202420    3025
             202430    1123
3.0-3.5      202210    2804
             202220    2473
             202230     915
             202310    2664
             202320    2329
             202330     839
             202410    2482
             202420    2203
             202430     708
3.5-4.0      202210    2981
             202220    2609
             202230     873
             202310    2927
             202320    2558
         

In [8]:
#print("This represents discontinued rate for each college and GPA group for Fall, Spring, and Summer terms.")

# Step 1: Classify terms into Fall, Spring, and Summer
def classify_semester(term):
    if str(term).endswith('10'):
        return 'Fall'
    elif str(term).endswith('20'):
        return 'Spring'
    elif str(term).endswith('30'):
        return 'Summer'

#Add a new column for semester type
outcome_df['Semester'] = outcome_df['term'].apply(classify_semester)

#Group by GPA_Group and term, calculate count of encrypt_id and sum of discontinued
disc_by_gpa = outcome_df.groupby(['GPA_Group', 'term', 'college_et', 'Semester']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})

#Calculate discontinued rate
disc_by_gpa['disc_rate'] = (disc_by_gpa['discontinued'] / disc_by_gpa['encrypt_id'] * 100).round(2)

#Reset index to make the DataFrame easy to read
disc_by_gpa = disc_by_gpa.reset_index()

#disc_pivot = disc_by_gpa.pivot_table(values='disc_rate', index='college_et', columns='GPA_Group')
#print("Discontinued rates by GPA Group and term:")
#disc_pivot
#disc_pivot3 = disc_by_gpa.pivot_table(values='disc_rate', index='GPA_Group', columns='Semester')
#print("Discontinued rates by GPA Group and Semester:")
#disc_pivot3

In [9]:
disc_pivot = disc_by_gpa.pivot_table(values='disc_rate', index=['Semester','college_et'], columns='GPA_Group').astype('int')
print("Discontinued rates by College and Semester not regarding high school/college groups:")
disc_pivot

Discontinued rates by College and Semester not regarding high school/college groups:


GPA_Group            2.0 or less  2.0-3.0  3.0-3.5  3.5-4.0
Semester college_et                                        
Fall     BU                   39        7        4        5
         ED                   47        8        4        2
         FA                   44       10        2        1
         FR                   43        6        3        2
         LA                   51        9        6        6
         SM                   43        8        4        2
Spring   BU                   52        8        5        6
         ED                   55        9        4        4
         FA                   58       10        5        3
         FR                   49        6        3        2
         LA                   54       19       20       33
         SM                   60       12        4        4
Summer   BU                   32        4        7        9
         ED                   33        4        1        1
         FA                   40        4        3        3
         FR                   29        3        3        4
         LA                   33        4        2        3
         SM                   30        6        5        9

In [10]:
#

# Assuming outcome_df is your DataFrame
type_counts = outcome_df['student_type_code'].value_counts()
print(type_counts)


#eliminate B and C


F    24267
T    12113
B     2551
R     2007
C      777
S       82
G       64
P        1
A        1
H        1
Name: student_type_code, dtype: int64


In [11]:
print('Discontinued Rates for All Students except UG-Dual Credit Continuing - HS, Early College Student, UG-Transient.')
# Check if any values in 'student_type_code' are in the list ['C', 'E', 'S'] and drop those rows
only_college_df = outcome_df.query("student_type_code not in ['C', 'S', 'E']")
only_college_df = only_college_df.dropna()
only_college_df = only_college_df.reset_index()

# Check if the rows were dropped and print the resulting DataFrame
print(only_college_df['student_type_code'])

Discontinued Rates for All Students except UG-Dual Credit Continuing - HS, Early College Student, UG-Transient.
0     R
1     F
2     F
3     T
4     F
     ..
74    T
75    F
76    F
77    F
78    F
Name: student_type_code, Length: 79, dtype: object


In [12]:

#only_college_df = only_college_df.groupby(['GPA_Group', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
#only_college_df['disc_rate'] = (only_college_df['discontinued'] / disc_by_classification['encrypt_id'] * 100).round(2)
#print(only_college_df['encrypt_id'])

#disc_pivot2 = only_college_df.pivot_table(values='disc_rate', index=['Semester','college_et'], columns='GPA_Group').astype('int')
#print("Discontinued rates by College and Semester regarding high school/college groups:")
#disc_pivot2

In [13]:
#diff = disc_pivot.compare(disc_pivot2)
#diff

In [14]:
#codes: 
#B - first time dual credit
#C - continuing dual credit student
#E - ? (banner delivered)